<a href="https://colab.research.google.com/github/josejustinos/projeto_alura/blob/main/projeto_alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import tkinter as tk
from tkinter import filedialog, messagebox
import fitz  # PyMuPDF
import re
import requests

# Definir suas chaves API e endpoint de acordo
API_KEY = 'SUA_GOOGLE_API_KEY'
ENDPOINT_URL = 'https://gemini.caminho_para_api'

# Função para analisar o texto do PDF com o Google Gemini
def analisa_pdf_conteudo(texto):
    # Solicitação à API do Google
    headers = {'Authorization': 'Bearer ' + API_KEY}
    payload = { 'texto': texto }
    response = requests.post(ENDPOINT_URL, headers=headers, json=payload)
    # Analisa a resposta e retorna resultados
    # Esta etapa é um placeholder, você precisa substituir com a chamada real
    return response.json()

# Função para extrair informação com base em um padrão regex
def extrai_informacao(regex, texto):
    match = re.search(regex, texto, re.IGNORECASE)
    return match.group(0) if match else "Informação não encontrada"

# Função para buscar as informações desejadas no texto
def buscar_informacoes(texto):
    padroes = {
        'numero_vagas': r'\b(\d+)\s+vagas\b',
        'grau_escolaridade': r'(Ensino Médio|Ensino Superior|Graduação|Pós-graduação|Doutorado|Mestrado)',
        'data_inicial': r'\b(\d{2}/\d{2}/\d{4})\b',
        'data_final': r'\b(\d{2}/\d{2}/\d{4})\b',
        'valor_inscricao': r'R\$\s?[0-9]+(?:,[0-9]{2})?',
        'local_inscricao': r'(https?://[^\s]+)'
    }
    resultados = { chave: extrai_informacao(regex, texto) for chave, regex in padroes.items() }
    return resultados

# Função para carregar um arquivo PDF
def carregar_pdf():
    caminho_arquivo = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
    if caminho_arquivo:  # Verifica se um arquivo foi selecionado
        with fitz.open(caminho_arquivo) as pdf:
            texto = ""
            for pagina in pdf:
                texto += pagina.get_text()
            # Google Gemini não é acessível, assim estamos simulando a busca por palavras
            if "inscrição para concurso público" in texto or "vagas para concurso público" in texto:
                informacoes = buscar_informacoes(texto)
                for chave, valor in informacoes.items():
                    print(f"{chave}: {valor}")
                opcao_usuario()
            else:
                print("Não foram encontrados concursos públicos nesse documento PDF.")
                opcao_usuario()
    else:
        print("Nenhum arquivo foi selecionado.")

# Função para exibir as opções ao usuário após a análise
def opcao_usuario():
    resposta = messagebox.askyesno("PDF Analisado", "Deseja analisar outro arquivo PDF?")
    if resposta:
        carregar_pdf()
    else:
        root.destroy()

# Criação da interface gráfica
root = tk.Tk()
root.withdraw()  # Esconder a janela principal

carregar_pdf()  # Iniciar escolhendo um arquivo PDF

TclError: no display name and no $DISPLAY environment variable